The main focus of this script is to create clean datasets for easy use 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [2]:
df = pd.read_csv("/home/elsherif/Desktop/Thesis/ViewPython/data/Train Data/rooftop.csv")
df.info()
# List of numeric columns that may have commas
numeric_cols = ["Unemployment_Rate", "Average_Age","employed"]

# Replace commas and convert to float
for col in numeric_cols:
    df[col] = df[col].str.replace(",", ".").astype(float)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14059 entries, 0 to 14058
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   tile                         14059 non-null  object 
 1   total_rooftops               14059 non-null  int64  
 2   rooftops_without_solar       14059 non-null  int64  
 3   square_meters_with_solar_m2  14059 non-null  float64
 4   panel_area_m2                14059 non-null  float64
 5   tile_centroid_lat            14059 non-null  float64
 6   tile_centroid_lon            14059 non-null  float64
 7   district_number              14059 non-null  int64  
 8   year                         14059 non-null  int64  
 9   Unemployment_Rate            14059 non-null  object 
 10  Average_Age                  14059 non-null  object 
 11  Elderly_Population           14059 non-null  float64
 12  Young_Population             14059 non-null  float64
 13  Total_Population

In [3]:
tiles_with_2020 = df[df["year"] == 2020]["tile"].unique()
all_tiles = df["tile"].unique()
tiles_missing_2020 = [t for t in all_tiles if t not in tiles_with_2020]

print("Tiles missing 2020:", tiles_missing_2020)


Tiles missing 2020: ['tile_r54_c10', 'tile_r54_c22']


In [4]:
df_clean = df[~df["tile"].isin(tiles_missing_2020)].copy()
df = df_clean
df = df.drop(columns=["Number_of_Houses"])


In [5]:
le = LabelEncoder()
df['tile_encoded'] = le.fit_transform(df['tile'])
num_tiles = df['tile_encoded'].nunique()


In [6]:
df.head()

,tile,total_rooftops,rooftops_without_solar,square_meters_with_solar_m2,panel_area_m2,tile_centroid_lat,tile_centroid_lon,district_number,year,Unemployment_Rate,Average_Age,Elderly_Population,Young_Population,Total_Population,employed,tile_encoded
0,tile_r0_c0,7,5,529.887,101.361,48.111706,11.444507,20,2012,3.8,42.9,10627.0,7361.0,47949,64.9,0
1,tile_r0_c1,9,9,0.000,0.000,48.114705,11.444507,20,2012,3.8,42.9,10627.0,7361.0,47949,64.9,1
2,tile_r0_c10,90,80,3339.949,201.642,48.141687,11.444507,21,2012,3.2,42.2,13608.0,11579.0,69295,66.2,2
3,tile_r0_c11,97,90,2715.933,343.005,48.144684,11.444507,21,2012,3.2,42.2,13608.0,11579.0,69295,66.2,3
4,tile_r0_c12,42,39,5222.084,75.094,48.147681,11.444507,21,2012,3.2,42.2,13608.0,11579.0,69295,66.2,4


In [7]:
# lag_features = [1]  # lag 1 and 2 years

# for lag in lag_features:
#     df[f'panel_area_lag{lag}'] = df.groupby('tile_encoded')['panel_area_m2'].shift(lag)

df = df.sort_values(["tile_encoded", "year"])

df["panel_area_lag1"] = (
    df.groupby("tile_encoded")["panel_area_m2"].shift(1)
)

# df["years_since_prev"] = (
#     df.groupby("tile_encoded")["year"].diff()
# )

# Load PV price data
pv_price_df = pd.read_csv("/home/elsherif/Desktop/Thesis/ViewPython/data/price_pv.csv")
pv_price_dict = dict(zip(pv_price_df['Year'], pv_price_df['Price_per_kWp_EUR']))

df["pv_price"] = df["year"].map(pv_price_dict)

df_model = df.dropna(subset=['panel_area_lag1', 'pv_price'])


In [8]:
df_model

,tile,total_rooftops,rooftops_without_solar,square_meters_with_solar_m2,panel_area_m2,tile_centroid_lat,tile_centroid_lon,district_number,year,Unemployment_Rate,Average_Age,Elderly_Population,Young_Population,Total_Population,employed,tile_encoded,panel_area_lag1,pv_price
12494,tile_r0_c0,7,7,0.000,0.000,48.111706,11.444507,20,2006,5.7,42.6,9505.0,7243.0,44993,65.6,0,14.890,5440.0
3130,tile_r0_c0,7,6,158.590,1.170,48.111706,11.444507,20,2009,4.6,42.7,10286.0,7240.0,46490,64.9,0,0.000,3800.0
0,tile_r0_c0,7,5,529.887,101.361,48.111706,11.444507,20,2012,3.8,42.9,10627.0,7361.0,47949,64.9,0,1.170,2150.0
4696,tile_r0_c0,6,5,156.775,4.786,48.111706,11.444507,20,2015,3.5,42.9,10953.0,7588.0,50257,65.5,0,101.361,1550.0
7796,tile_r0_c0,7,6,157.732,102.425,48.111706,11.444507,20,2018,2.6,43.2,10913.0,7429.0,49898,65.7,0,4.786,1350.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,tile_r9_c9,125,113,4051.710,189.448,48.138690,11.484931,21,2012,3.2,42.2,13608.0,11579.0,69295,66.2,1563,341.494,2150.0
9361,tile_r9_c9,127,117,4095.074,300.374,48.138690,11.484931,21,2018,2.7,42.4,14366.0,12638.0,74625,66.4,1563,189.448,1350.0
3129,tile_r9_c9,132,125,2251.183,173.403,48.138690,11.484931,21,2020,3.8,42.3,14631.0,13257.0,77301,66.5,1563,300.374,1280.0
7795,tile_r9_c9,124,110,4882.512,301.224,48.138690,11.484931,21,2022,3.2,41.9,14852.0,14062.0,80309,66.6,1563,173.403,1550.0


In [9]:
df_model.to_csv("final_model.csv", index=False)


In [10]:
df_model.info

<bound method DataFrame.info of              tile  total_rooftops  rooftops_without_solar  \
12494  tile_r0_c0               7                       7   
3130   tile_r0_c0               7                       6   
0      tile_r0_c0               7                       5   
4696   tile_r0_c0               6                       5   
7796   tile_r0_c0               7                       6   
...           ...             ...                     ...   
1565   tile_r9_c9             125                     113   
9361   tile_r9_c9             127                     117   
3129   tile_r9_c9             132                     125   
7795   tile_r9_c9             124                     110   
12493  tile_r9_c9             124                     112   

       square_meters_with_solar_m2  panel_area_m2  tile_centroid_lat  \
12494                        0.000          0.000          48.111706   
3130                       158.590          1.170          48.111706   
0                  